# Notebook examples for Chapter 7
# Supervised Classification Part 2
Continuing on the subject of supervised
classification, Chapter 7  begins with a discussion  of
post classification processing methods to
improve  results on the basis of contextual
information, after which  attention is turned to statistical
procedures for evaluating classification accuracy and for making
quantitative comparisons between different classifiers. 
As examples of  _ensembles_  of
classifiers, the _adaptive boosting_
technique is examined, applying it in particular to improve the generalization
accuracy of neural networks, and  the _random forest_ classifier, an ensemble of _binary  decision trees_ is also described. The remainder of the Chapter examines  more specialized forms of supervised image classification, namely as applied to   polarimetric SAR imagery, to data with  hyper-spectral resolution, and to   intermediate and high  resolution multispectral imagery using _convolutional neural networks_, _transfer learning_ and _semantic segmentation_.

In [2]:
import ee
ee.Authenticate()
ee.Initialize()
%matplotlib inline

Enter verification code:  4/1AeanS0bx4GyMoUwwnoWmRwDnA_jHhsjGDYAdxs-gnLBOGeLiZz-lExKK0dA



Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


## 7.1 Postprocessing
### 7.1.1 Majority filtering

### 7.1.2. Probabilistic label relaxation

In [ ]:
%run scripts/classify -a 6 -p [1,2,3,4,5] -L [10,10] -e 100 -P imagery/AST_20070501_pca.tif imagery/train.shp

In [ ]:
%run scripts/dispms -f imagery/AST_20070501_pca_classprobs.tif -e 2 -p [1,2,3] -d [400,400,300,300]

In [ ]:
run scripts/plr -i 3 imagery/AST_20070501_pca_classprobs.tif

In [ ]:
%run scripts/dispms -f imagery/AST_20070501_pca_class.tif -c -d [400,400,300,300] \
-F imagery/AST_20070501_pca_classprobs_plr.tif -C -D [400,400,300,300] \
#-s '/home/mort/LaTeX/new projects/CRC5/Chapter7/fig7_1.eps'

## 7.2 Evaluation and comparison of classification accuracy

### 7.2.1 Accuracy assessment

In [ ]:
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt

theta = 0.4
n = 2000
x = np.arange(600, 1000)
# pmf = probability mass function
plt.plot(x, binom.pmf(x, n, theta))

#plt.savefig('/home/mort/LaTeX/new projects/CRC4/Chapter7/fig7_2.eps')

In [ ]:
run scripts/ct imagery/AST_20070501_pca_NNet(Congrad).tst

### 7.2.2 Accuracy assessment on the GEE

In [ ]:
import ee
ee.Initialize()
# first 4 principal components of ASTER image
image = ee.Image('projects/ee-mortcanty/assets/CRC5/AST_20070501_pca') \
                                      .select(0,1,2,3)
# training data
table = ee.FeatureCollection('projects/ee-mortcanty/assets/CRC5/train')
# sample the image with the polygons to a feature  
# collection, rename the class id columns from strings 
# to integers, add a column of random numbers in [0,1]
trainTestData = image.sampleRegions(collection=table,
                                properties=['CLASS_ID'],
                                scale=15) \
 .remap(list(map(str,range(10))),list(range(10)),
                                           'CLASS_ID') \
 .randomColumn('rand',seed=12345) 
# filter on the random column to split into training and
# test feature collections in the ration of 2:1
trainData = trainTestData.filter(ee.Filter.lt('rand',0.67))
testData = trainTestData.filter(ee.Filter.gte('rand',0.67))
print('train pixels: %i'%trainData.size().getInfo())
print('test pixels:  %i'%testData.size().getInfo())
# train a (default) SVM classifier on training data   
classifier = ee.Classifier.libsvm()
trained = classifier.train(trainData,'CLASS_ID',
                                   image.bandNames()) 
# test the trained classifier with the test data
tested = testData.classify(trained)
# generate a confusion matrix with the 
# classified test data
cm = tested.errorMatrix('CLASS_ID','classification')
# and from it determine the accuracy and kappa 
print('accuracy: %f'%cm.accuracy().getInfo())
print('kappa:    %f'%cm.kappa().getInfo())

### 7.2.3 Crossvalidation on parallel architectures

In [ ]:
!ipcluster start -n 4 --daemonize

In [ ]:
run scripts/crossvalidate -p [1,2,3,4,5] -a 6 -e 10 -L [10,10] -e 1000 imagery/AST_20070501_pca.tif imagery/train.shp

### 7.2.4 Model comparison

In [ ]:
run scripts/classify -p [1,2,3,4,5] -a 6 -L [10,10] -e 500 imagery/AST_20070501_pca.tif imagery/train.shp

In [ ]:
run scripts/classify -p [1,2,3,4,5] -a 7 imagery/AST_20070501_pca.tif imagery/train.shp

In [ ]:
%run scripts/mcnemar imagery/AST_20070501_pca_SVM.tst \
imagery/AST_20070501_pca_Dnn(tensorflow).tst 

In [ ]:
run scripts/classify -p [1,2,3,4,5] -a 2  imagery/AST_20070501_pca.tif imagery/train.shp

In [ ]:
%run scripts/mcnemar imagery/AST_20070501_pca_Gausskernel.tst \
imagery/AST_20070501_pca_Dnn(tensorflow).tst

## 7.3 Ensembles
### 7.3.1 Adaptive boosting

In [ ]:
run scripts/adaboost -p [1,2,3,4,5]  -L 10 -n 50 imagery/AST_20070501_pca.tif  imagery/train.shp

In [ ]:
%run scripts/dispms -f imagery/AST_20070501_pca_class.tif -c \
-r  "['WATER', 'RAPESEED', 'SUGARBEET', 'SUBURBAN', 'INDUSTRIAL', 'CONIFEROUS', 'GRAIN', 'GRASSLAND', 'HERBIFEROUS', 'OPENCAST']" \
#-s '/home/mort/LaTeX/new projects/CRC5/Chapter7/fig7_x.eps'

### 7.3.2 Binary decision trees and random forests

In [ ]:
run scripts/classify -p [1,2,3,4,5] -a 8  imagery/AST_20070501_pca.tif imagery/train.shp

In [ ]:
%run scripts/dispms -f imagery/AST_20070501_pca_class.tif -c \
-r  "['WATER', 'RAPESEED', 'SUGARBEET', 'SUBURBAN', 'INDUSTRIAL', 'CONIFEROUS', 'GRAIN', 'GRASSLAND', 'HERBIFEROUS', 'OPENCAST']"

In [ ]:
%run scripts/mcnemar imagery/AST_20070501_pca_Gausskernel.tst \
imagery/AST_20070501_pca_RF.tst

## 7.4 Classification of polarimetric SAR imagery

## 7.5 Hyperspectral image analysis
### 7.5.1 Spectral mixture modeling
### 7.5.2 Unconstrained linear unmixing

In [ ]:
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly
import numpy as np
import matplotlib.pyplot as plt

inDataset = gdal.Open('imagery/97_radianz_registriert_voll',GA_ReadOnly)                       
cols = inDataset.RasterXSize
rows = inDataset.RasterYSize    
bands = inDataset.RasterCount
print(rows, cols, bands)

In [ ]:
Im = np.zeros((rows,cols,bands)) 
for b in range(bands):
    band = inDataset.GetRasterBand(b+1)
    Im[:,:,b] = band.ReadAsArray(0,0,cols,rows)  

In [ ]:
# spectral cut
plt.plot(Im[300,300,:])
#plt.savefig('/home/mort/LaTeX/new projects/CRC4/Chapter7/fig7_6.eps',bbox_inches='tight')

### 7.5.3 Intrinsic end-members and pixel purity

In [ ]:
run scripts/mnf.py imagery/97_radianz_registriert_voll

### 7.5.4 Anomaly detection: The RX algorithm

In [ ]:
run scripts/rx imagery/AST_20070501_pca.tif

In [ ]:
run scripts/dispms -f imagery/AST_20070501_pca_rx.tif -e 1 -d [600,600,400,400]

### 7.5.5 Anomaly detection: The kernel RX algorithm

In [ ]:
run scripts/krx -s 2000 imagery/AST_20070501_pca.tif

In [ ]:
%run scripts/dispms -f imagery/AST_20070501_pca_rx.tif -e 2 -d [600,600,250,250] \
-F imagery/AST_20070501_pca_krx.tif -E 2 -D [600,600,250,250] \
#-s '/home/mort/LaTeX/new projects/CRC5/Chapter7/fig7_9.eps'

## 7.6 Convolutional neural networks

### 7.6.1 Transfer learning

The examples in the text can be run on Google Colab with a GPU runtime. Use this link:

https://colab.research.google.com/drive/1aleipk9k16e5jAjR1FqcOl9NB7FUjLng

### 7.6.2 Semantic segmentation

These examples should be run on Google Colab with a GPU runtime. Use this link:

https://colab.research.google.com/drive/136YAVBZCwppz2SU-6Yxo1q2D2geyq0se